# Exercise: Sentiment guesser

## Understanding top-down & bottom-up approaches

Learning objectives
- Understand the difference between top-down (rule-based) and bottom-up (pattern-based) approaches in sentiment analysis.
- Implement a simple sentiment detection model using both approaches.
- Reflect on the advantages and limitations of each method.


In [ ]:
# Note that this exercise requires the following packages: 
#!pip3 install wordcloud matplotlib pandas numpy vaderSentiment seaborn

In [ ]:
## load packages
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from wordcloud import WordCloud

In [ ]:
GROUP_NAME = "GROUP AWESOME" # change this to your group name

In [ ]:
data = {"review": [
    "I absolutely love this product! It works great and is amazing.",
    "This is the worst purchase I have ever made. Totally terrible!",
    "It's okay, but I expected better. Not great but not bad either.",
    "Fantastic quality! Exceeded my expectations.",
    "I hate this so much. Worst thing ever!",
    "Pretty decent, could be improved but overall not bad.",
    "Superb experience! Highly recommend.",
    "Disappointed. Not what I expected at all.",
    "Well, it works... I guess.",  
    "I was excited to try this, but it completely failed my expectations.",  
    "It's a product."  
]}

df = pd.DataFrame(data)
df

### --- Step 1: Manual Sentiment Classification ---

With your team, please discuss whether you should enter _Positive_, _Negative_, or _Neutral_ for each review.
Please also rate your certainty for the label from 1 (very uncertain) to 5 (very certain).

In [ ]:
df["ManualSentiment"] = ["" for _ in range(len(df))]  # Placeholder for student input
df["Certainty"] = [0 for _ in range(len(df))]  # Placeholder for certainty rating

print("\n### Manual Sentiment Classification ###")
print("Please manually classify the following reviews as Positive, Negative, or Neutral.")
print("Also rate your certainty from 1 (very uncertain) to 5 (very certain).\n")

valid_sentiments = {"Positive", "Negative", "Neutral"}

for i, review in enumerate(df["review"]):
    while True:
        sentiment = input(f"Review: {review}\nYour Sentiment (Positive/Negative/Neutral): ").strip().capitalize()
        if sentiment in valid_sentiments:
            df.at[i, "ManualSentiment"] = sentiment
            break
        print("Invalid input. Please enter Positive, Negative, or Neutral.")
    
    while True:
        try:
            certainty = int(input("Certainty (1-5): "))
            if 1 <= certainty <= 5:
                df.at[i, "Certainty"] = certainty
                break
            else:
                print("Please enter a number between 1 and 5.")
        except ValueError:
            print("Invalid input. Please enter a number between 1 and 5.")


In [ ]:
df

# --- Step 2: Top-Down Approach ---
Use a predefined lexicon of positive and negative words. We make the following rule: 
- Positive Sentiment: If the number of positive words in the review exceeds the number of negative words, the sentiment is labeled as Positive.
- Negative Sentiment: If the number of negative words exceeds the number of positive words, the sentiment is labeled as Negative.
- Neutral Sentiment: If the number of positive words equals the number of negative words (or neither exceeds the other), the sentiment is labeled as Neutral.

Can *you* think of additional words that should be added to the list? how does adding more words change your results?

In [ ]:
positive_words = {"love", "great", "excellent", "amazing", "happy", "fantastic", "superb", "recommend"}
negative_words = {"worst", "terrible", "awful", "hate", "bad", "disappointed"}

def top_down_sentiment(text):
    words = text.lower().split()
    pos_count = sum(1 for word in words if word in positive_words)
    neg_count = sum(1 for word in words if word in negative_words)
    return "Positive" if pos_count > neg_count else "Negative" if neg_count > pos_count else "Neutral"

df["TopDownSentiment"] = df["review"].apply(top_down_sentiment)
df

# --- Step 3: Bottom-up approach ---
Use word frequency patterns to infer sentiment. 

The code is analyzing the frequency of words in a dataset of product reviews, based on the manually classified sentiment of each review, and visualizing the most common words using word clouds. Here's a breakdown of the operations:

1. **Combining and counting words in review**:
   - `all_words = " ".join(df["review"]).lower().split()`: 
     - This combines all reviews in the dataset into a single string (by joining them with spaces).
     - It then converts all text to lowercase to avoid case-sensitivity.
     - Finally, it splits the combined text into individual words.
   - `word_counts = Counter(all_words)`: 
     - This counts the frequency of each word in the dataset using Python's `Counter` class, which creates a dictionary-like object where the keys are the words and the values are the counts.
   
2. **Displaying the most frequent words**:
   - `for word, count in word_counts.most_common(20)`: 
     - This loops through the top 20 most frequent words in the dataset, where `most_common(20)` returns the 20 most frequent words and their counts.
     - It prints each word along with its frequency.

3. **Separating reviews by sentiment labels**:
   - `positive_reviews = df[df["ManualSentiment"] == 'Positive']['review']`: 
     - This creates a subset of the dataframe containing only the reviews that have been manually labeled as "Positive".
   - Similarly, it creates subsets for **negative** and **neutral** reviews by filtering based on the sentiment labels ("Negative" and "Neutral").
   
4. **Counting words for each sentiment category**:
   - The same process used for all reviews is repeated separately for positive, negative, and neutral reviews.
   - `positive_words = " ".join(positive_reviews).lower().split()`:
     - It joins the positive reviews into one long string, converts them to lowercase, and splits them into words.
   - `positive_word_counts = Counter(positive_words)`:
     - It counts the frequency of words in the positive reviews using the `Counter` class.
   - The same steps are repeated for **negative** and **neutral** reviews to count their words separately.

5. **Generating word clouds**:
   - `generate_wordcloud(word_counts, title)`: 
     - This function generates a word cloud visualization for a given set of word counts.
     - The `WordCloud` library is used to create the cloud, and `generate_from_frequencies` takes the word counts to create the word cloud.
     - The function then displays the word cloud using `plt.imshow()` from the `matplotlib` library.
     - `plt.title(title, fontsize=20)` adds a title to the word cloud.

6. **Displaying word clouds for each sentiment**:
   - The code then generates and displays word clouds for **positive**, **negative**, and **neutral** words using the `generate_wordcloud` function, with each word cloud being titled appropriately.


In [ ]:
# Split reviews into words, convert to lowercase, and count word frequencies
all_words = " ".join(df["review"]).lower().split()
word_counts = Counter(all_words)

# Count the most frequent words in the dataset (top 20)
print("Most common words in the dataset (top 20):")
for word, count in word_counts.most_common(20):
    print(f"{word}: {count}")

# Separate the reviews based on the sentiment labels
positive_reviews = df[df["ManualSentiment"] == 'Positive']['review']
negative_reviews = df[df["ManualSentiment"] == 'Negative']['review']
neutral_reviews = df[df["ManualSentiment"] == 'Neutral']['review']

# Get the word counts for positive reviews
positive_words = " ".join(positive_reviews).lower().split()
positive_word_counts = Counter(positive_words)

# Get the word counts for negative reviews
negative_words = " ".join(negative_reviews).lower().split()
negative_word_counts = Counter(negative_words)

# Get the word counts for neutral reviews
neutral_words = " ".join(neutral_reviews).lower().split()
neutral_word_counts = Counter(neutral_words)

# Function to generate word clouds
def generate_wordcloud(word_counts, title):
    wordcloud = WordCloud(width=800, height=400, background_color="white").generate_from_frequencies(word_counts)
    
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.title(title, fontsize=20)
    plt.show()

# Display word cloud for positive words
generate_wordcloud(positive_word_counts, "Positive words")

# Display word cloud for negative words
generate_wordcloud(negative_word_counts, "Negative words")

# Display word cloud for neutral words
generate_wordcloud(neutral_word_counts, "Neutral words")


In [ ]:
df

# --- Step 4: VADER Sentiment Analysis ---

In the final step, we will use a pre-trained model for sentiment classification. Since VADER relies on a predefined lexicon with sentiment scores assigned in advance, you could argue that it follows a top-down approach—applying predefined knowledge (the dictionary) to analyze new text. However, it also exhibits bottom-up characteristics because it computes the overall sentiment by aggregating individual word scores and adjusting based on syntactic rules (e.g., negation, intensifiers, punctuation).

So, it’s a bit of both:
- Top-down because it starts with a pre-built sentiment dictionary.
- Bottom-up because it builds sentiment from individual words and adjusts based on context.

In [ ]:
analyzer = SentimentIntensityAnalyzer()

def vader_sentiment(text):
    score = analyzer.polarity_scores(text)
    return "Positive" if score["compound"] > 0.05 else "Negative" if score["compound"] < -0.05 else "Neutral"

df["VADER_Sentiment"] = df["review"].apply(vader_sentiment)

In [ ]:
df

# --- Step 5: Agreement calculation  ---

What does this tell us about agreement with human annotations?

In [ ]:
def agreement_score(manual, predicted):
    return 1 if manual == predicted else 0

# apply the agreement score calculation
df["agreement_topdown"] = df.apply(lambda row: agreement_score(row["ManualSentiment"], row["TopDownSentiment"]), axis=1)
df["agreement_vader"] = df.apply(lambda row: agreement_score(row["ManualSentiment"], row["VADER_Sentiment"]), axis=1)

# calculate non-weighted agreement percentages
def non_weighted_agreement(agreement_col):
    return df[agreement_col].mean()

agreement_summary = {
    "top-down agreement": non_weighted_agreement("agreement_topdown"),
    "vader agreement": non_weighted_agreement("agreement_vader")
}

# --- Step 6: visualization of agreement scores ---

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

agreement_df = pd.DataFrame(list(agreement_summary.items()), columns=["SentimentMethod", "AgreementScore"])

# Plot agreement scores using Seaborn
plt.figure(figsize=(8, 5))
sns.barplot(x="SentimentMethod", y="AgreementScore", data=agreement_df, palette="Set2")


# Set plot labels and title
plt.ylim(0, 1)
plt.xlabel("Sentiment analysis method")
plt.ylabel("Agreement Score")
plt.title(f"Agreement between manual annotations and automated methods {GROUP_NAME}")

# Save the figure
plt.savefig(f"agreement_scores_{GROUP_NAME}.png")

# Show the plot
plt.show()


## FINAL: Show me the results!
Upload your final result: [here](https://amsuni-my.sharepoint.com/:f:/g/personal/a_c_kroon_uva_nl/EitpVGH7CXtKlnzRCye9zOoB5Fu-mIgISaA8IANncePrEQ?e=CQV5wX)
